In [1]:
import pandas as pd
import os

files = [file for file in os.listdir('.') if file.endswith('csv')]
model_names = list(set([file.split('_')[-1].split('.')[0] for file in files]))
print('all models', model_names)


# slatewise
# model_names = [
#     'MatrixFactorization', 'LogReg', 'NeuralClickModel','NeuralClickModelBest', 'InSlateAttention'
# ]

# sequencewise
model_names = [
    'FlatAttentionClickedOnly', 
    'InSlateAttentionSequencewiseGRU', 
    'FlattenedGRU' : 
]

all models ['FlatAttentionClickedOnly', 'NeuralClickModelBase', 'NeuralClickModel', 'InSlateAttentionSequencewiseGRU', 'NeuralClickModelBest', 'MatrixFactorization', 'FlattenedGRU', 'LogReg', 'FlatAttentionClickedOnly2LinearLnorm', 'InSlateAttention']


In [2]:
data = {}

dataset_rename = {'cw': 'ContentWise', 'rl4rs': 'RL4RS'}
metrics_rename = {'f1':'F1', 'accuracy': 'Accuracy', 'roc-auc':"ROC-AUC"}
embeddings_rename = {'svd':'SVD', 'neural':"nn.Embeddings", 'explicit':"Explicit"}

index = pd.MultiIndex.from_product(
    [["ContentWise", "RL4RS"], ["ROC-AUC", "F1", "Accuracy"]]
)

columns = pd.MultiIndex.from_product(
    [model_names, ["SVD", "nn.Embeddings", "Explicit"]]
)


df = pd.DataFrame(
    index=index,
    columns=columns,
)
data = df.to_dict()
data

{('FlatAttentionClickedOnly', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('FlatAttentionClickedOnly',
  'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan, ('ContentWise',
   'F1'): nan, ('ContentWise', 'Accuracy'): nan, ('RL4RS',
   'ROC-AUC'): nan, ('RL4RS', 'F1'): nan, ('RL4RS', 'Accuracy'): nan},
 ('FlatAttentionClickedOnly', 'Explicit'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('InSlateAttentionSequencewiseGRU', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('InSlateAttentionSequencewiseGRU',
  'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan, (

In [3]:
for file in files:
    dataset = dataset_rename[file.split('_')[0]]
    model = file.split('_')[-1].split('.')[0]
    data_dict = pd.read_csv(file).drop('Unnamed: 0', axis=1).set_index('embeddings').to_dict('index')
    for embedding, m in data_dict.items():
        for metric, score in m.items():
#             print(model, embeddings_rename[embedding], dataset, metrics_rename[metric], score )
            try:
                data[model, embeddings_rename[embedding]][dataset, metrics_rename[metric]] = score
            except:
                pass
        

In [4]:
df = pd.DataFrame(
    data,
    index=index,
    columns=columns,
).T.dropna(how='all')
df

ContentWise                      \
                                                  ROC-AUC        F1  Accuracy   
FlatAttentionClickedOnly        SVD              0.722628  0.308945  0.799244   
                                nn.Embeddings    0.712000  0.284926  0.866837   
                                Explicit              NaN       NaN       NaN   
InSlateAttentionSequencewiseGRU SVD              0.753003  0.334397  0.827983   
                                nn.Embeddings    0.750184  0.318957  0.875675   
                                Explicit              NaN       NaN       NaN   
FlattenedGRU                    SVD              0.736268  0.293388  0.861640   
                                nn.Embeddings    0.734139  0.306878  0.767766   
                                Explicit              NaN       NaN       NaN   

                                                  RL4RS                      
                                                ROC-AUC        F1  Accuracy  
FlatAttentionClickedOnly        SVD            0.926009  0.893666  0.858494  
                                nn.Embeddings  0.902031  0.935525  0.869447  
                                Explicit       0.897426  0.931149  0.864877  
InSlateAttentionSequencewiseGRU SVD            0.782082  0.812068  0.716963  
                                nn.Embeddings  0.790031  0.812148  0.720300  
                                Explicit       0.858991  0.846913  0.786507  
FlattenedGRU                    SVD            0.922482  0.884448  0.851094  
                                nn.Embeddings  0.920331  0.881862  0.847685  
                                Explicit       0.924105  0.885449  0.852352

In [5]:
df = df.style.format(decimal='.', precision=3).highlight_max(axis=0, props="font-weight:bold;")

In [6]:
df

In [7]:
print(df.to_latex(convert_css=True))

\begin{tabular}{llrrrrrr}
{} & {} & \multicolumn{3}{r}{ContentWise} & \multicolumn{3}{r}{RL4RS} \\
{} & {} & {ROC-AUC} & {F1} & {Accuracy} & {ROC-AUC} & {F1} & {Accuracy} \\
\multirow[c]{3}{*}{FlatAttentionClickedOnly} & SVD & 0.723 & 0.309 & 0.799 & \bfseries 0.926 & 0.894 & 0.858 \\
 & nn.Embeddings & 0.712 & 0.285 & 0.867 & 0.902 & \bfseries 0.936 & \bfseries 0.869 \\
 & Explicit & nan & nan & nan & 0.897 & 0.931 & 0.865 \\
\multirow[c]{3}{*}{InSlateAttentionSequencewiseGRU} & SVD & \bfseries 0.753 & \bfseries 0.334 & 0.828 & 0.782 & 0.812 & 0.717 \\
 & nn.Embeddings & 0.750 & 0.319 & \bfseries 0.876 & 0.790 & 0.812 & 0.720 \\
 & Explicit & nan & nan & nan & 0.859 & 0.847 & 0.787 \\
\multirow[c]{3}{*}{FlattenedGRU} & SVD & 0.736 & 0.293 & 0.862 & 0.922 & 0.884 & 0.851 \\
 & nn.Embeddings & 0.734 & 0.307 & 0.768 & 0.920 & 0.882 & 0.848 \\
 & Explicit & nan & nan & nan & 0.924 & 0.885 & 0.852 \\
\end{tabular}

